In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
# 구글드라이브 연결
from google.colab import drive
drive.mount('/gdrive')
# 데이터파일 불러오기

df = pd.read_csv('/gdrive/My Drive/3-2_machine_learning/image-10k.csv', header=None)

X = df.iloc[:, 1:].values    # 데이터샘플
y = df.iloc[:, 0].values      # 타겟레이블

STEP=25
score = np.zeros((STEP,))
def state(RANDOM_STATE):

  x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=RANDOM_STATE, stratify=y)

  import cv2

  hog = cv2.HOGDescriptor((28, 28), (14, 14), (7, 7), (7, 7), 9)
                                                       
  hog_x_train = np.zeros((5000,9*4*9),dtype="float64")
  hog_x_test = np.zeros_like(hog_x_train)

  for i in range(5000):
    hog_x_train[i]=hog.compute(x_train[i].reshape(28,28).astype(np.uint8)).squeeze()

  for i in range(5000):
    hog_x_test[i]=hog.compute(x_test[i].reshape(28,28).astype(np.uint8)).squeeze()
  from sklearn.preprocessing import StandardScaler
  
  hog_sc = StandardScaler()
  hog_sc.fit(hog_x_train)
  hog_x_train = hog_sc.transform(hog_x_train)
  hog_x_test = hog_sc.transform(hog_x_test)

  from sklearn.ensemble import VotingClassifier
  from sklearn.metrics import accuracy_score
  from sklearn.neural_network import MLPClassifier
  from sklearn import svm

  svm_1 = svm.SVC(kernel="rbf",gamma='auto',tol=1e-3,C=5e0,probability=True,random_state=RANDOM_STATE)
  svm_2 = svm.SVC(kernel="poly",C=10,probability=True,random_state=RANDOM_STATE)
  mlp_1 = MLPClassifier(hidden_layer_sizes=(100,),activation='relu',solver='adam',learning_rate="adaptive",
                        alpha=1e0,batch_size=500, max_iter=500,random_state=RANDOM_STATE)
  
  vo_clf = VotingClassifier(estimators=[('svm_1', svm_1),('svm_2',svm_2),('mlp_1',mlp_1)],  voting='hard')

  # Voting분류기 학습

  vo_clf.fit(hog_x_train,y_train)
  pred_y = vo_clf.predict(hog_x_test)
  print(RANDOM_STATE,'번째 state의 일치율', accuracy_score(y_test, pred_y))#Voting 분류기 정확도 0.8628
  
  score[RANDOM_STATE] = accuracy_score(y_test, pred_y)
  

Mounted at /gdrive


In [2]:
for i in range(STEP):
  state(i)

0 번째 state의 일치율 0.8774
1 번째 state의 일치율 0.8752
2 번째 state의 일치율 0.8732
3 번째 state의 일치율 0.8846
4 번째 state의 일치율 0.8742
5 번째 state의 일치율 0.8822
6 번째 state의 일치율 0.8746
7 번째 state의 일치율 0.8796
8 번째 state의 일치율 0.8794
9 번째 state의 일치율 0.8794
10 번째 state의 일치율 0.8784
11 번째 state의 일치율 0.8766
12 번째 state의 일치율 0.8758
13 번째 state의 일치율 0.8752
14 번째 state의 일치율 0.8744
15 번째 state의 일치율 0.8814
16 번째 state의 일치율 0.881
17 번째 state의 일치율 0.8774
18 번째 state의 일치율 0.8768
19 번째 state의 일치율 0.8806
20 번째 state의 일치율 0.8796
21 번째 state의 일치율 0.8736
22 번째 state의 일치율 0.8764
23 번째 state의 일치율 0.8768
24 번째 state의 일치율 0.875


In [4]:
print(f"random state 10개에 대한 acc의 평균은 {np.average(score):.4f}입니다")
print(score)
print(np.std(score))
print(np.max(score))

random state 10개에 대한 acc의 평균은 0.8776입니다
[0.8774 0.8752 0.8732 0.8846 0.8742 0.8822 0.8746 0.8796 0.8794 0.8794
 0.8784 0.8766 0.8758 0.8752 0.8744 0.8814 0.881  0.8774 0.8768 0.8806
 0.8796 0.8736 0.8764 0.8768 0.875 ]
0.002909999312714701
0.8846
